#Evaluation de divulgations sociales avec GPT

### Pré-requis pour faire tourner le code

In [1]:
#Installer les libraries python (pré-requis)

!pip install pyngrok
!pip install jiggybase
!ngrok config add-authtoken 2Vw6rI3erBLi6zJ2zVitHDLBR67_6LCGhxRwrpWCGciSryCSN
!pip install pydantic==1.10.12
# Import necessary libraries
from flask import Flask, render_template_string, request

#Connecter au cloud google drive
from google.colab import drive
drive.mount('/content/drive')

#Clé d'accès jiggybase : jgy-yjZCQzygEqNCuSyCftpNBzWkYimMmoDJMrPPgEVdLV

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 7.5 MB/s eta 0:00:00
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.0 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully uninstalled pydantic-1.10.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 

## Entreprises dont des rapports sont disponibles dans la base de connaissance de l'outil

In [ ]:
# Initialize JiggyBase
jb = jiggybase.JiggyBase()

# Set your organization and collection name
ORGANIZATION_NAME = 'elm09'
COLLECTION_NAME = 'Corporate social reports '

# Get a specific collection
collection_name = COLLECTION_NAME
col = jb.collection(collection_name)

# Get the document chunks
doc_chunks, next_index = col.get_doc_chunks(limit=100)  # Adjust limit as needed

# Print the titles of the documents
for doc in doc_chunks:
    # Extract the title from the metadata of the first chunk
    title = doc[0].metadata.author
    print(title)


Linamar
Magna International Inc.
CP Human Resources Department
CP
Exchange Income Corporation
Exchange Income Corporation
None
Alain Bédard, FCPA, FCA
Dr. Ajay Virmani
Cogeco
Shaw Communications Inc.
Rogers Communications Inc.
TELUS
BCE Inc.
Bell
Celestica
Nuvei Corporation
Philip Fayer
None
Shopify
Kinaxis Inc.
KINAXIS INC.
Enghouse
Constellation Brands, Inc.
OpenText
Lightspeed
None
Thomson Reuters
Boyd Group Services Inc.
Boyd Group Services Inc.
Boyd Group Services Inc.
Stantec, Inc.
Stantec
Badger
Badger Infrastructure Solutions Ltd.
Waste Connections
Toromont Industries Ltd.
Finning International Inc.
Chief Human Resources Officer (CHRO)
L. Scott Thomson
CCL Industries Inc.
SVP, Corporate Development & Sustainability
Brad Boyd, SVP, Corporate Resources
Methanex
Superior Plus Corp.
Superior Plus Corp.
Innergex Renewable Energy Inc.
Innergex Renewable Energy Inc.
INE
Northland Power Inc.
Northland Power
TransAlta Corporation
Emera Incorporated
Scott Balfour, President and CEO, Emer


### Site web


In [ ]:
#Faire tourner l'application web

from flask import Flask, render_template_string, request, url_for, redirect, flash
import os
import ast
import jiggybase
from werkzeug.utils import secure_filename
from jiggybase.models import PromptMessage


app = Flask(__name__, static_folder='/content/drive/MyDrive/Chat-LLMs - ESG : Social evaluation/Others')

app.config['UPLOAD_FOLDER'] = 'uploads/'
app.secret_key = "supersecretkey"

jb = jiggybase.JiggyBase()
ORGANIZATION_NAME = 'elm09'
COLLECTION_NAME = 'Corporate social reports '
collection = jb.collection(COLLECTION_NAME)
org = jb.get_org(ORGANIZATION_NAME)

with open("/content/drive/MyDrive/Chat-LLMs - ESG : Social evaluation/Prompt templates/Generic_base_prompt_template.txt", 'r') as questions_file:
    questions = [q.strip() for q in questions_file.readlines() if q.strip() != '-']

company_list = [
    "Richelieu Hardware Ltd",
    "Ritchie Bros. Auctioneers Incorporated",
    "Canada Goose Holdings, Inc.",
    "Gildan Activewear Inc.",
    "Spin Master Corp",
    "BRP, Inc.",
    "Aritzia, Inc.",
    "Canadian Tire Corporation",
    "Dollarama Inc.",
    "Uni-Select Inc.",
    "Sleep Country Canada",
    "Pet Valu Ltd.",
    "Jamieson Wellness Inc.",
    "Cameco Corporation",
    "Denison Mines Corp.",
    "NexGen Energy Ltd.",
    "Imperial Oil Limited",
    "Cenovus Energy Inc.",
    "Suncor Energy Inc.",
    "Precision Drilling Corporation",
    "Secure Energy Services Inc.",
    "Pason Systems Inc.",
    "Africa Oil Corp.",
    "Tourmaline Oil Corp.",
    "Advantage Energy Ltd",
    "Vermilion Energy Inc.",
    "MEG Energy Corp.",
    "Spartan Delta Corp",
    "Birchcliff Energy Ltd.",
    "Enerplus Corporation",
    "Athabasca Oil Corporation",
    "Freehold Royalties Ltd.",
    "Peyto Exploration & Development Corp.",
    "Crescent Point Energy Corp.",
    "Baytex Energy Corp.",
    "PrairieSky Royalty Ltd",
    "International Petroleum Corp. (British Columbia)",
    "Canadian Natural Resources Limited",
    "Whitecap Resources Inc.",
    "Tamarack Valley Energy Ltd.",
    "Topaz Energy Corp",
    "ARC Resources Ltd.",
    "NuVista Energy Ltd.",
    "Parex Resources Inc.",
    "Headwater Exploration Inc.",
    "Paramount Resources Ltd. Class A",
    "Parkland Corporation",
    "Pembina Pipeline Corporation",
    "Enbridge Inc.",
    "Keyera Corp.",
    "Gibson Energy Inc.",
    "TC Energy Corporation",
    "AltaGas Ltd.",
    "Ero Copper Corp.",
    "Capstone Copper Corp",
    "First Quantum Minerals Ltd.",
    "Lundin Mining Corporation",
    "Teck Resources Limited",
    "Filo Mining Corp.",
    "Lithium Americas Corp.",
    "Hudbay Minerals Inc",
    "Ivanhoe Mines Ltd",
    "Canfor Corporation",
    "Interfor Corporation",
    "West Fraser Timber Co. Ltd.",
    "Stella-Jones Inc.",
    "SSR Mining Inc",
    "Eldorado Gold Corporation",
    "Franco-Nevada Corporation",
    "Torex Gold Resources Inc.",
    "Alamos Gold Inc.",
    "K92 Mining, Inc.",
    "Centerra Gold Inc.",
    "Yamana Gold Inc.",
    "Agnico Eagle Mines Limited",
    "Osisko Mining Inc",
    "Equinox Gold Corp.",
    "Novagold Resources Inc",
    "Barrick Gold Corporation",
    "Dundee Precious Metals Inc.",
    "B2Gold Corp.",
    "IAMGOLD Corporation",
    "Lundin Gold Inc.",
    "Wesdome Gold Mines Ltd.",
    "Wheaton Precious Metals Corp",
    "Seabridge Gold Inc",
    "Sandstorm Gold Ltd.",
    "Osisko Gold Royalties Ltd",
    "Kinross Gold Corporation",
    "SilverCrest Metals, Inc.",
    "Fortuna Silver Mines Inc.",
    "First Majestic Silver Corp.",
    "Endeavour Silver Corp.",
    "MAG Silver Corp.",
    "Pan American Silver Corp.",
    "Labrador Iron Ore Royalty Corporation",
    "Algoma Steel Group Inc.",
    "Stelco Holdings, Inc.",
    "SNC-Lavalin Group Inc.",
    "Russel Metals Inc.",
    "Bank of Nova Scotia (Scotiabank)",
    "Bank of Montreal",
    "National Bank of Canada",
    "Canadian Imperial Bank of Commerce",
    "Laurentian Bank of Canada",
    "Toronto-Dominion Bank",
    "Royal Bank of Canada",
    "Canadian Western Bank",
    "EQB Inc",
    "IGM Financial Inc.",
    "Sprott Inc.",
    "Brookfield Asset Management Ltd.",
    "Onex Corporation",
    "CI Financial Corp.",
    "Home Capital Group Inc.",
    "goeasy Ltd.",
    "TMX Group Ltd.",
    "Element Fleet Management Corporation",
    "Manulife Financial Corporation",
    "Sun Life Financial Inc.",
    "Power Corporation of Canada",
    "Great-West Lifeco Inc.",
    "iA Financial Corporation Inc.",
    "Definity Financial Corp.",
    "Fairfax Financial Holdings Limited",
    "Trisura Group Ltd.",
    "Intact Financial Corporation",
    "Brookfield Business Partners LP",
    "Brookfield Infrastructure Partners L.P.",
    "Brookfield Renewable Partners LP",
    "Restaurant Brands International Inc",
    "MTY Food Group Inc.",
    "George Weston Limited",
    "Empire Co. Ltd.",
    "North West Company Inc.",
    "Saputo Inc.",
    "Premium Brands Holdings Corp",
    "Primo Water Corporation",
    "Nutrien Ltd.",
    "Sienna Senior Living Inc.",
    "Chartwell Retirement Residences",
    "BELLUS Health Inc.",
    "Bausch Health Companies Inc.",
    "Canopy Growth Corporation",
    "Tilray Brands, Inc.",
    "H&R Real Estate Investment Trust",
    "NorthWest Healthcare Properties Real Estate Investment Trust",
    "Dream Industrial Real Estate Investment Trust",
    "Granite Real Estate Investment Trust Stapled Units - 1 Granite RE Inc + 1 TU Granite REIT",
    "Boardwalk REIT",
    "InterRent Real Estate Investment Trust",
    "Killam Apartment Real Estate Investment Trust",
    "Canadian Apartment Properties Real Estate Investment Trust",
    "Allied Properties Real Estate Investment Trust",
    "CT Real Estate Investment Trust",
    "Choice Properties Real Estate Investment Trust",
    "Crombie Real Estate Investment Trust",
    "RioCan Real Estate Investment Trust",
    "First Capital Real Estate Investment Trust",
    "SmartCentres Real Estate Investment Trust",
    "Primaris Real Estate Investment Trust",
    "Tricon Residential Inc",
    "StorageVault Canada Inc.",
    "Colliers International Group Inc.",
    "FirstService Corp",
    "Altus Group Limited",
    "ATS Corporation",
    "ATCO Ltd.",
    "Fortis Inc.",
    "Hydro One Limited",
    "Emera Incorporated",
    "Capital Power Corporation",
    "TransAlta Corporation",
    "Algonquin Power & Utilities Corp.",
    "Canadian Utilities Limited",
    "Northland Power Inc.",
    "TransAlta Renewables, Inc.",
    "Innergex Renewable Energy Inc.",
    "Ballard Power Systems Inc.",
    "CAE Inc.",
    "Superior Plus Corp",
    "Winpak Ltd.",
    "Transcontinental Inc.",
    "Methanex Corporation",
    "CCL Industries Inc.",
    "Finning International Inc.",
    "Toromont Industries Ltd.",
    "Waste Connections, Inc.",
    "GFL Environmental Inc",
    "WSP Global Inc.",
    "Badger Infrastructure Solutions Ltd",
    "Stantec Inc",
    "Boyd Group Services Inc",
    "Thomson Reuters Corporation",
    "Park Lawn Corporation",
    "Lightspeed Commerce Inc",
    "Open Text Corporation",
    "Constellation Software Inc.",
    "Descartes Systems Group Inc.",
    "Enghouse Systems Limited",
    "Kinaxis, Inc.",
    "Dye & Durham Ltd.",
    "Shopify, Inc.",
    "CGI Inc.",
    "Converge Technology Solutions Corp.",
    "Nuvei Corp.",
    "BlackBerry Limited",
    "Celestica Inc.",
    "Bell Canada Entreprises Inc.",
    "TELUS Corporation",
    "Rogers Communications Inc.",
    "Shaw Communications Inc.",
    "Cogeco Communications Inc.",
    "Quebecor Inc.",
    "Bombardier Inc. Class B",
    "Cargojet Inc.",
    "TFI International Inc.",
    "Mullen Group Ltd.",
    "Westshore Terminals Investment Corporation",
    "Air Canada",
    "Exchange Income Corporation",
    "Canadian National Railway Company",
    "Canadian Pacific Railway Limited",
    "Magna International Inc.",
    "Linamar Corporation"
]

def submit_prompt(company, prompt_text):
    prompt_message = PromptMessage(content=prompt_text, role="user", position=1, extras=None)
    task = org.create_prompt_task(name=company.replace(" ", "_"), version=1, prompts=[prompt_message], type=None, description=f"{company}_report")
    response = collection._chat_completion(task.prompts, temperature=0, model="gpt-3.5-turbo")
    return response.choices[0].message.content

@app.route('/', methods=['GET', 'POST'])
def index():
    prompt = ''
    response = ''
    question_list = questions[:25]

    if request.method == 'POST':
        selected_company = request.form.get('company')
        print("Selected Company:", selected_company)
        selected_question = request.form.get('question')
        custom_question = request.form.get('user_prompt')



        if custom_question:
            prompt = custom_question.replace('...', f"about {selected_company}?").replace('X', selected_company)
        else:
            prompt = questions[questions.index(selected_question)].replace('...', f"about {selected_company}?").replace('X', selected_company)


        # if custom_question:
        #     prompt = custom_question.replace('...', f"about {selected_company}?")
        # else:
        #     prompt = questions[questions.index(selected_question)].replace('...', f"about {selected_company}?")

        response = submit_prompt(selected_company, prompt)
        print("Final Prompt:", prompt)

    return render_template_string('''
         <style>
    body {
        font-family: Arial, sans-serif;
        background: linear-gradient(to right, #2980b9, #6dd5fa, #ffffff);
        color: #333;
        margin: 0;
        padding: 0;
    }
    .container {
        max-width: 800px;
        margin: 50px auto;
        background: #fff;
        padding: 25px;
        border-radius: 8px;
        box-shadow: 0 0 15px rgba(0, 0, 0, 0.1);
    }
    select, input[type="submit"], textarea {
        display: block;
        width: 100%;
        padding: 10px;
        margin-top: 10px;
        border: 1px solid #ccc;
        border-radius: 4px;
    }
    input[type="submit"] {
        background: #007BFF;
        color: #fff;
        cursor: pointer;
    }
    input[type="submit"]:hover {
        background: #0056b3;
    }
    .response {
        margin-top: 20px;
    }
    .response p {
        background: #f4f4f4;
        padding: 10px;
        border: 1px solid #ddd;
        border-radius: 4px;
    }
    .logos {
        display: flex;
        justify-content: space-between;  # <-- Modified this line to space the logos
        align-items: center;
        margin-top: 50px;
    }
    .logos img {
        max-width: 150px;
    }
</style>
<div class="container">
    <h1>Evaluating social disclosures with GPT</h1>
    <form method="POST">
        <label for="company">Select Company:</label>
        <select name="company">
            {% for company in company_list %}
            <option value="{{ company }}">{{ company }}</option>
            {% endfor %}
        </select>
        <label for="question">Select Standard Question:</label>
        <select name="question">
            {% for question in question_list %}
            <option value="{{ question }}">{{ question }}</option>
            {% endfor %}
        </select>
        <label for="user_prompt">Or Enter Your Own Question:</label>
        <textarea name="user_prompt" rows="4"></textarea>
        <input type="submit" value="Submit">
    </form>
    <div class="response">
        <strong>Prompt:</strong>
        <p>{{ prompt }}</p>
        <strong>Response:</strong>
        <p>{{ response }}</p>
    </div>
    <div class="logos">
        <img src="{{ url_for('static', filename='cropped-Logo-Algora-udm.png') }}" alt="Algora Lab">
        <img src="{{ url_for('static', filename='finml.webp') }}" alt="FINML">
    </div>
</div>
''', company_list=company_list, question_list=question_list, prompt=prompt, response=response)

if __name__ == '__main__':
    !pip install flask-ngrok
    from flask_ngrok import run_with_ngrok
    run_with_ngrok(app)
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://7dab-35-231-246-200.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


###Analyse a grande échelle


In [ ]:
import json
import datetime
import jiggybase
from jiggybase.models import PromptMessage
import os


import jiggybase.collection
import jiggybase.org



print("Available companies:")
for i, company in enumerate(company_list, 1):
    print(f"{i}. {company}")

selected_indexes = input("\nEnter the numbers (comma-separated) of the companies you want to evaluate: ").split(",")
companies_to_evaluate = [company_list[int(index)-1] for index in selected_indexes]

# 2. Allow the user to specify the file path of the prompt template
default_path = "/content/drive/MyDrive/Chat-LLMs - ESG : Social evaluation/Prompt templates/Generic_base_prompt_template.txt"
template_path = input(f"\nEnter the path of the prompt template (default: {default_path}): ") or default_path


drive_directory = "/content/drive/MyDrive/Chat-LLMs - ESG : Social evaluation/Answers/"
default_output_file = f"{len(companies_to_evaluate)}_{datetime.date.today()}.json"
output_file_name_input = input(f"\nName the output file (default: {default_output_file}): ") or default_output_file
output_file_path = os.path.join(drive_directory, output_file_name_input)



# Read the template file
with open(template_path, 'r') as file:
    template = file.read()

# Parse the content based on the delimiter '\n-'
prompts_template = template.split("\n-")
master_dict = {}

for company in companies_to_evaluate:
    for i, prompt in enumerate(prompts_template):
        key = f"{company}_{i+1}"  # Added an underscore for better readability
        master_dict[key] = prompt.replace("X", company)


# Get responses and save them to master_dict
for key, prompt_content in master_dict.items():
    # Extract company name and prompt number from the key
    company_name = ''.join([i for i in key if not i.isdigit()])

    # Create a PromptMessage for each prompt
    prompt_message = PromptMessage(
        content=prompt_content,
        role="user",
        position=1,
        extras=None,
    )

    # Create a task for each prompt
    task_name = key.replace(" ", "_")
    task_description = f"{company_name}_report"
    task = org.create_prompt_task(
        name=task_name,
        version=1,
        prompts=[prompt_message],
        description=task_description
    )

    # Get responses from the prompt task
    temperature = 0
    model = "gpt-3.5-turbo"
    response = collection._chat_completion(task.prompts, temperature=temperature, model=model)


    # Print the responses during the loop
    if response.choices:
        response_content = response.choices[0].message.content
        print(f"For prompt {key}:\n{response_content}\n{'-'*50}")

        master_dict[key] = {
            'prompt': prompt_content,
            'response': response_content
        }
    else:
        print(f"No response for prompt {key}.\n{'-'*50}")
        master_dict[key] = {
            'prompt': prompt_content,
            'response': None
        }

# Add 'pred_label' key to each entry in master_dict based on the response
for key, value in master_dict.items():
    if value['response'] and value['response'].strip().lower().startswith("yes"):
        value['pred_label'] = 1
    else:
        value['pred_label'] = 0

# Save the master_dict to the specified output file
with open(output_file_path, 'w') as json_file:
    json.dump(master_dict, json_file, indent=4)


Available companies:
1. Richelieu Hardware Ltd
2. Ritchie Bros. Auctioneers Incorporated
3. Canada Goose Holdings, Inc.
4. Gildan Activewear Inc.
5. Spin Master Corp
6. BRP, Inc.
7. Aritzia, Inc.
8. Canadian Tire Corporation
9. Dollarama Inc.
10. Uni-Select Inc.
11. Sleep Country Canada
12. Pet Valu Ltd.
13. Jamieson Wellness Inc.
14. Cameco Corporation
15. Denison Mines Corp.
16. NexGen Energy Ltd.
17. Imperial Oil Limited
18. Cenovus Energy Inc.
19. Suncor Energy Inc.
20. Precision Drilling Corporation
21. Secure Energy Services Inc.
22. Pason Systems Inc.
23. Africa Oil Corp.
24. Tourmaline Oil Corp.
25. Advantage Energy Ltd
26. Vermilion Energy Inc.
27. MEG Energy Corp.
28. Spartan Delta Corp
29. Birchcliff Energy Ltd.
30. Enerplus Corporation
31. Athabasca Oil Corporation
32. Freehold Royalties Ltd.
33. Peyto Exploration & Development Corp.
34. Crescent Point Energy Corp.
35. Baytex Energy Corp.
36. PrairieSky Royalty Ltd
37. International Petroleum Corp. (British Columbia)
38. Ca

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install jiggybase
!pip install pydantic==1.10.12


# Import Required Libraries
import os
import logging
from google.colab import drive
import jiggybase
import json


# Initialize Logger for Debugging
logging.basicConfig(level=logging.INFO)

#  Mount Google Drive
# def mount_google_drive():
#     """Mount Google Drive to access data folder."""
#     drive.mount('/content/drive')

# mount_google_drive()

#  Initialize JiggyBase
def initialize_jiggybase(organization_name, collection_name):
    """
    Initialize Jiggybase and return the JiggyBase instance, organization, and collection.

    Parameters:
    - organization_name (str): The name of the JiggyBase organization.
    - collection_name (str): The name of the collection within the organization.

    Returns:
    - tuple: A tuple containing JiggyBase instance, organization, and collection.
    """
    jb = jiggybase.JiggyBase()
    org = jb.get_org(organization_name)
    collection = org.collection(collection_name)
    return jb, org, collection

ORGANIZATION_NAME = 'elm09'
COLLECTION_NAME = 'Social sample'
jb, org, collection = initialize_jiggybase(ORGANIZATION_NAME, COLLECTION_NAME)

# Validate and Set Data Folder Path
def set_data_folder_path(folder_path):
    """
    Validates and returns the folder path.

    Parameters:
    - folder_path (str): The path to the data folder.

    Returns:
    - str: The validated folder path, or None if invalid.
    """
    if os.path.exists(folder_path):
        return folder_path
    else:
        logging.error(f"The folder path {folder_path} does not exist.")
        return None

FOLDER_PATH = '/content/drive/MyDrive/Chat-LLMs - ESG : Social evaluation/Sample'
folder_path = set_data_folder_path(FOLDER_PATH)

# File Upload Operations
def upload_file(collection, filename):
    """
    Upload a file to the Jiggybase collection.

    Parameters:
    - collection (Collection): The Jiggybase collection object.
    - filename (str): The name of the file to upload.
    """
    logging.info(f'Uploading {filename}')
    try:
        collection.upsert_file(filename)
    except Exception as e:
        logging.error(f'Error on {filename}: {e}')

def upload_directory(collection, dirname):
    """
    Upload all files from a directory to the Jiggybase collection.

    Parameters:
    - collection (Collection): The Jiggybase collection object.
    - dirname (str): The name of the directory containing files to upload.
    """
    for filename in os.listdir(dirname):
        full_path = os.path.join(dirname, filename)
        upload_file(collection, full_path)

# Main Execution
def main(folder_path, collection):
    """Main function to upload files or directories."""
    if os.path.isfile(folder_path):
        upload_file(collection, folder_path)
    elif os.path.isdir(folder_path):
        upload_directory(collection, folder_path)
    else:
        logging.error(f"The path provided is not a valid file or directory: {folder_path}")

# Execute the main function
if __name__ == "__main__":
    main(folder_path, collection)

# Create Task and Generate Response
def create_prompt_task(organization, prompt_name, prompt_content, company_name, task_version=1):
    """
    Creates a task for a given prompt using JiggyBase and returns it.

    Parameters:
    - organization: The JiggyBase organization object.
    - prompt_name (str): The name of the prompt.
    - prompt_content (str): The content of the prompt.
    - company_name (str): The name of the company the prompt is for.
    - task_version (int): The version of the task.

    Returns:
    - task: The created task object.
    """
    from jiggybase.models import PromptMessage
    prompt_message = PromptMessage(
        content=prompt_content,
        role="user",
        position=1,
        extras=None
    )

    task = organization.create_prompt_task(
        name=prompt_name.replace(" ", "_"),
        version=task_version,
        prompts=[prompt_message],
        type=None,
        description=f"{company_name}_report"
    )

    return task

# Initialize List of Companies
def initialize_companies():
    """
    Returns the list of companies for which prompts will be generated.

    Returns:
    - list: List of company names.
    """
    return [
        "Northland Power",
        "Methanex",
        "Cargojet",
        "TFI International Inc",
        "Mullen Group Ltd.",
        "Magna International Inc.",
        "Linamar",
        "Nuvei",
        "TELUS",
        "Nutrien"
    ]

# Load Template
def load_template(template_file_path):
    """
    Loads the prompt template from a file.

    Parameters:
    - template_file_path (str): The path to the template file.

    Returns:
    - str: Content of the template file.
    """
    try:
        with open(template_file_path, 'r') as file:
            return file.read()
    except FileNotFoundError:
        logging.error(f"Template file {template_file_path} not found.")
        return None

#  Create Prompts
def create_prompts(companies, template):
    """
    Creates prompts for each company based on the template.

    Parameters:
    - companies (list): List of company names.
    - template (str): Prompt template.

    Returns:
    - dict: A dictionary containing prompts for each company.
    """
    prompts_template = template.split("\n-")
    master_dict = {}

    for company in companies:
        for i, prompt in enumerate(prompts_template):
            key = f"{company}_{i+1}"
            master_dict[key] = prompt.replace("X", company)

    return master_dict

# Save Prompts to File
def save_prompts_to_file(prompts_dict, output_file_path):
    """
    Saves the generated prompts to a JSON file.

    Parameters:
    - prompts_dict (dict): A dictionary containing generated prompts.
    - output_file_path (str): File path to save the JSON.
    """
    try:
        with open(output_file_path, 'w') as json_file:
            json.dump(prompts_dict, json_file, indent=4)
    except FileNotFoundError:
        logging.error(f"Could not write to file {output_file_path}")

# Main Execution for Prompting and Response Generation
if __name__ == '__main__':
    companies = initialize_companies()
    template_path = '/content/drive/MyDrive/Chat-LLMs - ESG : Social evaluation/Prompt templates/Template_Social_v0.txt'
    template = load_template(template_path)


    if template:
        prompts_dict = create_prompts(companies, template)
        output_file_path = '/content/drive/MyDrive/Chat-LLMs - ESG : Social evaluation/Prompt templates/prompt_dictionary_extractor.txt'
        save_prompts_to_file(prompts_dict, output_file_path)


In [ ]:
import json
import jiggybase
from jiggybase.models import PromptMessage
import jiggybase.collection
import jiggybase.org
import os


# List of companies
companies = [
    "Northland Power",
    "Methanex",
    "Cargojet",
    "TFI International Inc",
    "Mullen Group Ltd.",
    "Magna International Inc.",
    "Linamar",
    "Nuvei",
    "TELUS",
    "Nutrien"
]

# Read the template file
template_path = '/content/drive/MyDrive/Chat-LLMs - ESG : Social evaluation/Prompt templates/Template_Social_v0.txt'
with open(template_path, 'r') as file:
    template = file.read()

# Parse the content based on the delimiter '\n-'
prompts_template = template.split("\n-")
master_dict = {}

for company in companies:
    for i, prompt in enumerate(prompts_template):
        key = f"{company}_{i+1}"  # Added an underscore for better readability
        master_dict[key] = prompt.replace("X", company)

# Initialize JiggyBase
COLLECTION_NAME = 'Corporate social reports '
collection = jiggybase.JiggyBase().collection(COLLECTION_NAME)

# Get responses and save them to master_dict
for key, prompt_content in master_dict.items():
    # Extract company name and prompt number from the key
    company_name = ''.join([i for i in key if not i.isdigit()])

    # Create a PromptMessage for each prompt
    prompt_message = PromptMessage(
        content=prompt_content,
        role="user",
        position=1,
        extras=None,
    )

    # Create a task for each prompt
    task_name = key.replace(" ", "_")
    task_description = f"{company_name}_report"
    task = org.create_prompt_task(
        name=task_name,
        version=1,
        prompts=[prompt_message],
        description=task_description
    )

    # Get responses from the prompt task
    temperature = 0
    model = "gpt-3.5-turbo"
    response = collection._chat_completion(task.prompts, temperature=temperature, model=model)

    # Print the responses during the loop
    if response.choices:
        response_content = response.choices[0].message.content
        print(f"For prompt {key}:\n{response_content}\n{'-'*50}")

        master_dict[key] = {
            'prompt': prompt_content,
            'response': response_content
        }
    else:
        print(f"No response for prompt {key}.\n{'-'*50}")
        master_dict[key] = {
            'prompt': prompt_content,
            'response': None
        }

# Add 'pred_label' key to each entry in master_dict based on the response
for key, value in master_dict.items():
    if value['response'] and value['response'].strip().lower().startswith("yes"):
        value['pred_label'] = 1
    else:
        value['pred_label'] = 0

# Save the master_dict to a file once it's done
file_path = '/content/drive/MyDrive/Chat-LLMs - ESG : Social evaluation/Prompt templates/prompt_dictionary_temp.json'
with open(file_path, 'w') as json_file:
    json.dump(master_dict, json_file, indent=4)

For prompt Northland Power_1:
Yes, Northland Power has organizational objectives for EDI (Equality, Diversity, and Inclusion). These objectives are mentioned in their 2021 Sustainability Report, where they state their commitment to ensuring their people feel respected, included, and empowered to contribute to the advancement, growth, and success of Northland. They also mention promoting diversity by ensuring women make up 30% of their Board and Executive Management Team.
--------------------------------------------------
For prompt Northland Power_2:
Based on the information provided in the 2021 Sustainability Report of Northland Power, there is no explicit mention of Equality, Diversity, and Inclusion (EDI) themes in their mission statement or organizational values.
--------------------------------------------------
For prompt Northland Power_3:
Yes, Northland Power has an official EDI policy or declaration. According to the information provided in the 2021 Sustainability Report, Nort